# Scenario 6

Run this notebook to calculate the real values based on the set FileName in the ReadPatterson file. This is controlled for the settings of Scenario **1**

First run the notebook to retrieve the specific project network information

In [1]:
#%run ./Thesis_Copy-ReadPatterson.ipynb

In [2]:
# Import specific packages
import math
from ipykernel import kernelapp as app

## SCENARIO 6 details

Scenario 6 is a simulation set up, which has a correct average project ahead of performance indication since the project finishes ahead of schedule.

The parameters for the real duration are set as:

- minimum = duration * 0.9
- maximum = duration * 1.3

### Create the project
First we create a new actDur list that we will use for the PM metrics

In [3]:
metricinput = actDur

# Create dummy values for 'Cost Value' and 'Avg Cost'
metricinput['Cost Value'] = round(Decimal(0),5)
metricinput['Avg Cost'] = round(Decimal(0),5)

# Assign cost values as a new column to metrics for all activities except the dummy-variables
for i in range(1,nbAct-1,1):
        # Set the min and max value to calculate the random number
        minvalue = round(Decimal(50),5)
        maxvalue = round(Decimal(250),5)
        randomNumb = random.randint(minvalue,maxvalue)
        randomNumb = round(Decimal(randomNumb),5)
        # Assign the random number to the column 'Cost Value'
        metricinput.loc[i,'Cost Value'] = randomNumb
        costvalue = metricinput.loc[i,'Cost Value']
        # Get the activity duration
        activityDur = float(metricinput.loc[i,'Activity Duration'])
        activityDur = round(Decimal(activityDur),5)
        # Assign the 'Avg cost' value as the planned Cost value divided by the planned Activity duration
        metricinput.loc[i,'Avg Cost'] = round(costvalue / activityDur,5)

In [4]:
# Save the outcome to an Excel file for comparison
#metricinput.to_excel("thesis_metricplanned.xlsx")

## Calculate the real duration of the project

We are running the simulation for Scenario 1.

In [5]:
def GetRealDuration(duration):
    global RealDuration, cumdistr_step1,cumdistr_step2,cumdistr_step3,cumdistr_step4,cumdistr_step5
    
    minimum = duration * 0.9
    maximum = duration * 1.3
    
    step1 = int(minimum)
    step2 = int(minimum + ((maximum-minimum)/4))
    step3 = int(step2 + ((maximum-minimum)/4))
    step4 = int(step3 + ((maximum-minimum)/4))
    step5 = int(maximum)

    bracket1 = 0.1
    bracket2 = 0.2
    bracket3 = 0.5
    bracket4 = 0.75
    
    cumdistr_lst = []
    for x in range(1,1001,1):
        randomNumb = random.randint(0,100)/100
        if randomNumb < bracket1:
            cumdistr_lst.append(step1)
        elif randomNumb < bracket2:
            cumdistr_lst.append(step2)
        elif randomNumb < bracket3:
            cumdistr_lst.append(step3)
        elif randomNumb < bracket4:
            cumdistr_lst.append(step4)
        else:
            cumdistr_lst.append(step5)
    
    cumdistr_lst = pd.DataFrame(cumdistr_lst)
    cumdistr_lst.columns = ['Duration']
    cumdistr_step1 = round((int(cumdistr_lst[cumdistr_lst.Duration == step1].count())/1000) * 100, 2)
    cumdistr_step2 = round((int(cumdistr_lst[cumdistr_lst.Duration == step2].count())/1000) * 100, 2)
    cumdistr_step3 = round((int(cumdistr_lst[cumdistr_lst.Duration == step3].count())/1000) * 100, 2)
    cumdistr_step4 = round((int(cumdistr_lst[cumdistr_lst.Duration == step4].count())/1000) * 100, 2)
    cumdistr_step5 = round((int(cumdistr_lst[cumdistr_lst.Duration == step5].count())/1000) * 100, 2)
    cumdistr = (cumdistr_step1, 
                cumdistr_step1 + cumdistr_step2, 
                cumdistr_step1 + cumdistr_step2 + cumdistr_step3, 
                cumdistr_step1 + cumdistr_step2 + cumdistr_step3 + cumdistr_step4, 
                cumdistr_step1 + cumdistr_step2 + cumdistr_step3 + cumdistr_step4 + cumdistr_step5)
    
    randomNumb = random.randint(0,100)
    RealDuration = 0
    if randomNumb > cumdistr_step1:
        if randomNumb > cumdistr_step2:
            if randomNumb > cumdistr_step3:
                if randomNumb > cumdistr_step4:
                    RealDuration = step4
                else:
                    RealDuration = step5
            else:
                RealDuration = step3
        else:
            RealDuration = step2
    else:
        RealDuration = step1
    
    if RealDuration == 0:
        RealDuration = 1
        
    return RealDuration      

Calculate the real duration for each activity in the project

In [6]:
# Create 3 new columns in the metricinput dataframe for the Real values
metricinput['Real Duration'] = Decimal(0.000)
metricinput['Real Cost'] = Decimal(0.000)
metricinput['Avg Real Cost'] = Decimal(0.000)

# For each activity define the 3 new values and add them to the dataframe
for i in range(1,nbAct-1,1):
    # Get the Planned Duration
    duration = metricinput.loc[i,'Activity Duration']
    # Run the function GetRealDuration, and use Planned Duration as input value
    metricinput.loc[i,'Real Duration'] = GetRealDuration(duration)
    # Get the Planned Cost Value
    costvalue = Decimal(metricinput.loc[i,'Cost Value'])
    # Convert the Real Duration value to a decimal and add a small portion to avoid division by zero
    realdurvalue = Decimal(metricinput.loc[i,'Real Duration'])+Decimal(0.000000001)
    # Get the Planned Duration
    actdurvalue = float(duration)
    actdurvalue = round(Decimal(actdurvalue),5)
    # Get the Planned Average Cost
    avgcostvalue = Decimal(metricinput.loc[i,'Avg Cost'])
    # Calculate the Real Cost and Avg Real Cost and add this to the dataframe
    metricinput.loc[i,'Real Cost'] = round(costvalue + ((realdurvalue - actdurvalue) * avgcostvalue),5)
    metricinput.loc[i,'Avg Real Cost'] = round(costvalue / realdurvalue,5)
    
metricinput

,Activity ID,Activity Duration,Activity Start,Activity End,Cost Value,Avg Cost,Real Duration,Real Cost,Avg Real Cost
0,0,0,1,0,0.00000,0.00000,0,0,0
1,1,10,1,10,123.00000,12.30000,5,61.50000,24.60000
2,2,1,1,1,177.00000,177.00000,1,177.00000,177.00000
3,3,3,1,3,70.00000,23.33333,1,23.33334,70.00000
4,4,5,1,5,111.00000,22.20000,1,22.20000,111.00000
5,5,5,2,6,117.00000,23.40000,1,23.40000,117.00000
6,6,6,11,16,88.00000,14.66667,1,14.66665,88.00000
7,7,2,1,2,168.00000,84.00000,1,84.00000,168.00000
8,8,3,1,3,194.00000,64.66667,1,64.66666,194.00000
9,9,6,1,6,224.00000,37.33333,2,74.66668,112.00000


In [7]:
# Save the outcome to an Excel file for comparison
#metricinput.to_excel("thesis_metricinput.xlsx")

## Create a new Gantt chart for Real Duration

### Create the longest path based on Real Duration

In [8]:
longest_path_real = 0

In [9]:
def calculateLongestPathReal(act_index_new, current_length_new):
        global longest_path_real
        realduration = metricinput.loc[act_index_new,'Real Duration']
        pathlength_new = current_length_new + realduration
        actualSuc = nbSuc.loc[act_index_new]['Number of Successors']

        if  actualSuc == 0:
            return pathlength_new
        
        else:
            
            for s in range(0,actualSuc,1):

                actualSucpath_new = actSuc.loc[act_index_new][s]
                
                if actualSucpath_new != -1:
                
                    path_new = calculateLongestPathReal(actualSucpath_new,pathlength_new)
          
                    longest_path_real = max(path_new, longest_path_real)
    
            return longest_path_real

In [10]:
calculateLongestPathReal(0,0)
ev_longestpath = calculateLongestPathReal(0,0)

### Create the corrected gantt chart

Create the baseline Gantt chart, but add any additional columns if the longest_path_real > longest_path

In [11]:
maxlongestpath = max(int(longest_path), int(longest_path_real))
maxlongestpath = int(maxlongestpath)
dev_longestpath = longest_path - longest_path_real
dev_longestpath = int(dev_longestpath)

# Make sure the already created ganttchart gets more columns in case the real duration is longer
if dev_longestpath < 0:
    dev_longestpath = abs(dev_longestpath)
    for j in range(0,dev_longestpath,1):
        columnName = int(longest_path + (j+1))
        ganttchart.insert(columnName, columnName, 0)

ganttchart = ganttchart.astype(float)
ganttchart

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Save the outcome in the Excel files
#ganttchart.to_excel("thesis_ganttchart_adjusted.xlsx")

### Create the Gantt Chart based on Real values

In [13]:
height_new = int(nbAct)
width_new = int(maxlongestpath+1)
gantt_real = pd.DataFrame(0, index=range(height_new), columns=range(width_new))

metricinput['Real Start'] = 1
metricinput['Real End'] = 0 + metricinput['Real Duration']

collength_new = len(actPrePivot.columns)

for i in range(0,nbAct,1):
    for j in range(1,collength_new,1):
        if actPrePivot.loc[i][j] == 0:
            metricinput.loc[i,'Real Start'] = 1
            metricinput.loc[i,'Real End'] = 0 + metricinput.loc[i,'Real Duration']

        elif actPrePivot.loc[i][j] > 0:
            # Set the predecessor value to 'value'
            value_new = actPrePivot.loc[i][j] # = predecessor
            endvalue_new = metricinput.loc[value_new]['Real End']
            metricinput.loc[i,'Real Start'] = endvalue_new + 1
            metricinput.loc[i,'Real End'] = endvalue_new + metricinput.loc[i,'Real Duration']
            
for i in range(0,nbAct,1):
    for j in range(0,width_new,1):
        startValue_new = metricinput.loc[i,'Real Start']
        endValue_new = metricinput.loc[i,'Real End']
        if j >= startValue_new and j <= endValue_new:
            gantt_real.loc[i][j] = 1

In [14]:
gantt_real = gantt_real.astype(float)
gantt_real

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Save the outcome in the Excel files
#ganttchart.to_excel("thesis_ganttchart_realduration.xlsx")
#metricinput.to_excel("thesis_metricinput_real.xlsx")

### Create Planned Cost gantt chart

In [16]:
height_planned = int(nbAct)
width_planned = int(maxlongestpath+1)
gantt_planned = pd.DataFrame(float(0), index=range(height_planned), columns=range(width_planned))

for i in range(0,nbAct,1):
    for j in range(1,width_planned,1):
        ganttchartvalue = ganttchart.loc[i][j]
        ganttchartvalue = round(float(ganttchartvalue),5)
        
        prevganttchartvalue = gantt_planned.loc[i][j-1]
        prevganttchartvalue = round(float(prevganttchartvalue),5)
        
        avgcostvalue = metricinput.loc[i]['Avg Cost']
        avgcostvalue = round(float(avgcostvalue),5)
        
        calcValue = prevganttchartvalue + ( avgcostvalue * ganttchartvalue)
        calcValue = round(float(calcValue),5)
        
        gantt_planned.loc[i][j] = float(calcValue)
        
sum_row = {col: gantt_planned[col].sum() for col in gantt_planned}
# Turn the sums into a DataFrame with one row with an index of 'Total':
sum_df = pd.DataFrame(sum_row, index=["Total PV"])
# Now append the row:
gantt_planned = gantt_planned.append(sum_df)

In [17]:
gantt_planned = gantt_planned.astype(float)
gantt_planned

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.0,12.30000,24.60000,36.90000,49.20000,61.50000,73.80000,86.10000,98.40000,110.70000,...,123.00000,123.00000,123.00000,123.00000,123.00000,123.00000,123.00000,123.00000,123.00000,123.00000
2,0.0,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,...,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000,177.00000
3,0.0,23.33333,46.66666,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999,...,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999,69.99999
4,0.0,22.20000,44.40000,66.60000,88.80000,111.00000,111.00000,111.00000,111.00000,111.00000,...,111.00000,111.00000,111.00000,111.00000,111.00000,111.00000,111.00000,111.00000,111.00000,111.00000
5,0.0,0.00000,23.40000,46.80000,70.20000,93.60000,117.00000,117.00000,117.00000,117.00000,...,117.00000,117.00000,117.00000,117.00000,117.00000,117.00000,117.00000,117.00000,117.00000,117.00000
6,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,88.00002,88.00002,88.00002,88.00002,88.00002,88.00002,88.00002,88.00002,88.00002,88.00002
7,0.0,84.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,...,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000,168.00000
8,0.0,64.66667,129.33334,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001,...,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001,194.00001
9,0.0,37.33333,74.66666,111.99999,149.33332,186.66665,223.99998,223.99998,223.99998,223.99998,...,223.99998,223.99998,223.99998,223.99998,223.99998,223.99998,223.99998,223.99998,223.99998,223.99998


In [18]:
#gantt_planned.to_excel("thesis_gantt_planed.xlsx")

### Create Real cost gantt chart

In [49]:
height_realcost = int(nbAct)
width_realcost = int(maxlongestpath+1)

gantt_realcost = pd.DataFrame(float(0), index=range(height_realcost), columns=range(width_realcost))

for i in range(0,nbAct,1):
    for j in range(1,width_realcost,1):
        ganttchartvalue = gantt_real.loc[i][j]
        ganttchartvalue = round(float(ganttchartvalue),5)
        
        prevganttchartvalue = gantt_realcost.loc[i][j-1]
        prevganttchartvalue = round(float(prevganttchartvalue),5)
       
        avgcostvalue = metricinput.loc[i]['Avg Cost']
        avgcostvalue = round(float(avgcostvalue),5)

        calcValue = prevganttchartvalue + ( avgcostvalue * ganttchartvalue)
        calcValue = round(float(calcValue),5)
        
        gantt_realcost.loc[i][j] = float(calcValue)

sum_row = {col: gantt_realcost[col].sum() for col in gantt_realcost}
# Turn the sums into a DataFrame with one row with an index of 'Total':
sum_df = pd.DataFrame(sum_row, index=["Total AC"])
# Now append the row:
gantt_realcost = gantt_realcost.append(sum_df)

In [50]:
gantt_realcost

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,24.60,49.2,73.80,98.4,123.0,123.0,123.0,123.0,123.0,...,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0
2,0.0,177.00,177.0,177.00,177.0,177.0,177.0,177.0,177.0,177.0,...,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0
3,0.0,70.00,70.0,70.00,70.0,70.0,70.0,70.0,70.0,70.0,...,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0
4,0.0,111.00,111.0,111.00,111.0,111.0,111.0,111.0,111.0,111.0,...,111.0,111.0,111.0,111.0,111.0,111.0,111.0,111.0,111.0,111.0
5,0.0,0.00,117.0,117.00,117.0,117.0,117.0,117.0,117.0,117.0,...,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0
6,0.0,0.00,0.0,0.00,0.0,0.0,88.0,88.0,88.0,88.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0
7,0.0,168.00,168.0,168.00,168.0,168.0,168.0,168.0,168.0,168.0,...,168.0,168.0,168.0,168.0,168.0,168.0,168.0,168.0,168.0,168.0
8,0.0,194.00,194.0,194.00,194.0,194.0,194.0,194.0,194.0,194.0,...,194.0,194.0,194.0,194.0,194.0,194.0,194.0,194.0,194.0,194.0
9,0.0,112.00,224.0,224.00,224.0,224.0,224.0,224.0,224.0,224.0,...,224.0,224.0,224.0,224.0,224.0,224.0,224.0,224.0,224.0,224.0


In [21]:
#gantt_realcost.to_excel("thesis_gantt_realcost.xlsx")

### Create EV gantt chart

In [22]:
height_ev = int(nbAct)
width_ev = int(maxlongestpath+1)

gantt_ev = pd.DataFrame(float(0), index=range(height_ev), columns=range(width_ev))

for i in range(0,nbAct,1):
    for j in range(1,width_ev,1):
        ganttchartvalue = gantt_real.loc[i][j]
        ganttchartvalue = round(float(ganttchartvalue),5)
            
        prevganttchartvalue = gantt_ev.loc[i][j-1]
        prevganttchartvalue = round(float(prevganttchartvalue),5)
               
        costvalue = metricinput.loc[i]['Cost Value']
        costvalue = round(float(costvalue),5)
        
        realdurvalue = metricinput.loc[i,'Real Duration']
        realdurvalue = round(float(realdurvalue),5)
        realdurvalue = realdurvalue + 0.0000000001
        
        avgcostvalue = float(costvalue) / float(realdurvalue)
        
        calcValue = prevganttchartvalue + ( avgcostvalue * ganttchartvalue)
        calcValue = round(float(calcValue),5)
        
        gantt_ev.loc[i][j] = float(calcValue)
        
sum_row = {col: gantt_ev[col].sum() for col in gantt_ev}
# Turn the sums into a DataFrame with one row with an index of 'Total':
sum_df = pd.DataFrame(sum_row, index=["Total EV"])
# Now append the row:
gantt_ev = gantt_ev.append(sum_df)

In [23]:
gantt_ev

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,24.60,49.2,73.80,98.4,123.0,123.0,123.0,123.0,123.0,...,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0
2,0.0,177.00,177.0,177.00,177.0,177.0,177.0,177.0,177.0,177.0,...,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0
3,0.0,70.00,70.0,70.00,70.0,70.0,70.0,70.0,70.0,70.0,...,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0
4,0.0,111.00,111.0,111.00,111.0,111.0,111.0,111.0,111.0,111.0,...,111.0,111.0,111.0,111.0,111.0,111.0,111.0,111.0,111.0,111.0
5,0.0,0.00,117.0,117.00,117.0,117.0,117.0,117.0,117.0,117.0,...,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0,117.0
6,0.0,0.00,0.0,0.00,0.0,0.0,88.0,88.0,88.0,88.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0
7,0.0,168.00,168.0,168.00,168.0,168.0,168.0,168.0,168.0,168.0,...,168.0,168.0,168.0,168.0,168.0,168.0,168.0,168.0,168.0,168.0
8,0.0,194.00,194.0,194.00,194.0,194.0,194.0,194.0,194.0,194.0,...,194.0,194.0,194.0,194.0,194.0,194.0,194.0,194.0,194.0,194.0
9,0.0,112.00,224.0,224.00,224.0,224.0,224.0,224.0,224.0,224.0,...,224.0,224.0,224.0,224.0,224.0,224.0,224.0,224.0,224.0,224.0


In [24]:
#gantt_ev.to_excel("thesis_gantt_ev.xlsx")

In [43]:
project_metric = pd.DataFrame()
PV = gantt_planned.iloc[[32]]
EV = gantt_ev.iloc[[32]]
AC = gantt_realcost.iloc[[32]]
project_metric = project_metric.append(PV)
project_metric = project_metric.append(AC)
project_metric = project_metric.append(EV)
project_metric

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
Total PV,0.0,966.79286,1650.78572,2067.77858,2463.10477,2858.43096,3286.05715,3489.40001,3557.95715,3612.76429,...,4160.20001,4204.60001,4249.00001,4293.40001,4313.00001,4332.60001,4352.20001,4371.80001,4391.40001,4411.00001
Total AC,0.0,1960.35000,3452.70000,3616.55000,3780.40000,3943.50000,4112.60000,4177.40000,4242.20000,4307.00000,...,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000
Total EV,0.0,1960.35000,3452.70000,3616.55000,3780.40000,3943.50000,4112.60000,4177.40000,4242.20000,4307.00000,...,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000,4411.00000


### Calculate ES (Earned Schedule)

In [46]:
width_es = int(maxlongestpath+1)
ES = pd.DataFrame(0, index=["Total ES"], columns=range(width_es))
ES = ES.astype(float)
ES

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
Total ES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
longest_spi = int(maxlongestpath+1)

for i in range(1,longest_spi,1):
    EVvalue = project_metric.loc['Total EV'][i]
    curPVvalue = project_metric.loc['Total PV'][i]
    PVvaluemax = project_metric.loc['Total PV'][maxlongestpath]
    
    if EVvalue > PVvaluemax:
        ES.loc['Total ES'][i] = longest_path
    
    else:
    
        for j in range(1,longest_spi,1):
            PVvalue = project_metric.loc['Total PV'][j]
            if EVvalue < PVvalue:
                if j > 1:
                    PVprev = project_metric.loc['Total PV'][j-1]
                    colvalue = float(j)-1
                    ESvalue = (colvalue) + ((EVvalue - PVprev)/(PVvalue-PVprev))
                    ES.loc['Total ES'][i] = ESvalue
                    break
                else:
                    PVprev = float(0)
                    ESvalue = 0 + ((EVvalue - PVprev)/(PVvalue-PVprev))
                    ES.loc['Total ES'][i] = ESvalue
                
project_metric = project_metric.append(ES)
project_metric

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
Total PV,0.0,966.792860,1650.785720,2067.778580,2463.104770,2858.430960,3286.057150,3489.400010,3557.957150,3612.764290,...,4160.200010,4204.600010,4249.000010,4293.400010,4313.000010,4332.600010,4352.200010,4371.800010,4391.400010,4411.000010
Total AC,0.0,1960.350000,3452.700000,3616.550000,3780.400000,3943.500000,4112.600000,4177.400000,4242.200000,4307.000000,...,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000
Total EV,0.0,1960.350000,3452.700000,3616.550000,3780.400000,3943.500000,4112.600000,4177.400000,4242.200000,4307.000000,...,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000
Total ES,0.0,2.742373,6.819517,9.069073,11.292811,13.161642,15.918919,17.387387,18.846847,20.693877,...,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999


In [51]:
longest_spi = int(maxlongestpath+1)
SPI = pd.DataFrame(float(0), index=["SPI_t"], columns=range(longest_spi))
SPI = SPI.astype(float)
SPI

for j in range(1,longest_spi,1):
    if j == 26:
        SPI.loc['SPI_t'][j] = float(1)
    else:
        ESvalue = float(project_metric.loc['Total ES'][j])
        if ESvalue == 0:
            SPIvalue = float(j) / (ESvalue+0.00000000000001)
            SPI.loc['SPI_t'][j] = SPIvalue
        else:
            SPIvalue = ESvalue / float(j)
            SPI.loc['SPI_t'][j] = SPIvalue

project_metric = project_metric.append(SPI)
project_metric

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
Total PV,0.0,966.792860,1650.785720,2067.778580,2463.104770,2858.430960,3286.057150,3489.400010,3557.957150,3612.764290,...,4160.200010,4204.600010,4249.000010,4293.400010,4313.000010,4332.600010,4352.200010,4371.800010,4391.400010,4411.000010
Total AC,0.0,1960.350000,3452.700000,3616.550000,3780.400000,3943.500000,4112.600000,4177.400000,4242.200000,4307.000000,...,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000
Total EV,0.0,1960.350000,3452.700000,3616.550000,3780.400000,3943.500000,4112.600000,4177.400000,4242.200000,4307.000000,...,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000,4411.000000
Total ES,0.0,2.742373,6.819517,9.069073,11.292811,13.161642,15.918919,17.387387,18.846847,20.693877,...,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999,25.999999
SPI_t,0.0,2.742373,3.409758,3.023024,2.823203,2.632328,2.653153,2.483912,2.355856,2.299320,...,1.529412,1.444444,1.368421,1.300000,1.238095,1.181818,1.130435,1.083333,1.040000,1.000000


In [29]:
#project_metric.to_excel("checkSPIvalues.xlsx")

In [52]:
output = project_metric.iloc[-1]
output = pd.DataFrame(output)

In [75]:
spi = output.T
spi

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
SPI_t,0.0,2.742373,3.409758,3.023024,2.823203,2.632328,2.653153,2.483912,2.355856,2.29932,...,1.529412,1.444444,1.368421,1.3,1.238095,1.181818,1.130435,1.083333,1.04,1.0


In [132]:
cols = ev_longestpath
spi_finish = spi[spi.columns[:cols]]
spi_finish

,0,1,2,3,4,5,6,7,8,9,10
SPI_t,0.0,2.742373,3.409758,3.023024,2.823203,2.632328,2.653153,2.483912,2.355856,2.29932,2.4


Calculate the 5% interval

In [133]:
percinterval = ev_longestpath / 20
percinterval

Decimal('0.55')

### Calculate the 5% intervals

In [163]:
# create a length of the range for 10% - 100%
length = 11

# Create a data frame where the final values can be added to
final_metric = pd.DataFrame(float(0), index=["SPI_t"], columns=range(1,20,1))

# set the interval (longest_path for EV divided by 20, to create 5% intervals)
interval = float(percinterval)

for i in range(1,21,1):
    interval_step = interval * float(i)
    interval_step = round(interval_step, 3)
    
    RoundUp = math.ceil(interval_step)
    RoundUpprev = RoundUp - 1
    ratioToCalc = interval_step / RoundUp
    
    if RoundUp == ev_longestpath:
        SPIvalue = spi_finish.loc['SPI_t'][RoundUp-1]
        OldSPIvalue = spi_finish.loc['SPI_t'][RoundUp-2]
        NewSPIvalue = OldSPIvalue + (SPIvalue - OldSPIvalue) * ((RoundUp-(RoundUpprev)) -(RoundUp-interval_step))
    else:
        SPIvalue = spi_finish.loc['SPI_t'][RoundUp]
        OldSPIvalue = spi_finish.loc['SPI_t'][RoundUpprev]
        NewSPIvalue = OldSPIvalue + (SPIvalue - OldSPIvalue) * ((RoundUp-(RoundUpprev)) -(RoundUp-interval_step))
    
    final_metric.loc['SPI_t',i] = NewSPIvalue

Final_SPI = final_metric
Final_SPI

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
SPI_t,1.508305,2.809112,3.176173,3.332412,3.119708,2.963078,2.853176,2.746853,2.641872,2.642741,2.644691,2.551609,2.464704,2.394273,2.341722,2.310627,2.334558,2.389932,2.344626,2.4
